# Creating the Dataset
### Raw Data

In [1]:
from pathlib import Path
import mne_bids
import os 

In [2]:
BIDS_ROOT = Path('ds004504-download')

In [4]:
mne_bids.print_dir_tree(BIDS_ROOT)

|ds004504-download\
|--- .gitattributes
|--- CHANGES
|--- README
|--- dataset_description.json
|--- participants.json
|--- participants.tsv
|--- .datalad\
|------ config
|--- derivatives\
|------ sub-001\
|--------- eeg\
|------------ sub-001_task-eyesclosed_eeg.set
|------ sub-002\
|--------- eeg\
|------------ sub-002_task-eyesclosed_eeg.set
|------ sub-003\
|--------- eeg\
|------------ sub-003_task-eyesclosed_eeg.set
|------ sub-004\
|--------- eeg\
|------------ sub-004_task-eyesclosed_eeg.set
|------ sub-005\
|--------- eeg\
|------------ sub-005_task-eyesclosed_eeg.set
|------ sub-006\
|--------- eeg\
|------------ sub-006_task-eyesclosed_eeg.set
|------ sub-007\
|--------- eeg\
|------------ sub-007_task-eyesclosed_eeg.set
|------ sub-008\
|--------- eeg\
|------------ sub-008_task-eyesclosed_eeg.set
|------ sub-009\
|--------- eeg\
|------------ sub-009_task-eyesclosed_eeg.set
|------ sub-010\
|--------- eeg\
|------------ sub-010_task-eyesclosed_eeg.set
|------ sub-011\
|----

In [6]:
subjects = mne_bids.get_entity_vals(root=BIDS_ROOT, entity_key='subject')

In [9]:
subjects

['001',
 '002',
 '003',
 '004',
 '005',
 '006',
 '007',
 '008',
 '009',
 '010',
 '011',
 '012',
 '013',
 '014',
 '015',
 '016',
 '017',
 '018',
 '019',
 '020',
 '021',
 '022',
 '023',
 '024',
 '025',
 '026',
 '027',
 '028',
 '029',
 '030',
 '031',
 '032',
 '033',
 '034',
 '035',
 '036',
 '037',
 '038',
 '039',
 '040',
 '041',
 '042',
 '043',
 '044',
 '045',
 '046',
 '047',
 '048',
 '049',
 '050',
 '051',
 '052',
 '053',
 '054',
 '055',
 '056',
 '057',
 '058',
 '059',
 '060',
 '061',
 '062',
 '063',
 '064',
 '065',
 '066',
 '067',
 '068',
 '069',
 '070',
 '071',
 '072',
 '073',
 '074',
 '075',
 '076',
 '077',
 '078',
 '079',
 '080',
 '081',
 '082',
 '083',
 '084',
 '085',
 '086',
 '087',
 '088']

In [10]:
subject_paths = []
for subject in subjects:
    path = mne_bids.BIDSPath(
        subject=subject,
        task='eyesclosed',
        suffix='eeg',
        datatype='eeg',
        extension='.set',
        root=BIDS_ROOT
    )
    subject_paths.append(path)

subject_paths


[BIDSPath(
 root: ds004504-download
 datatype: eeg
 basename: sub-001_task-eyesclosed_eeg.set),
 BIDSPath(
 root: ds004504-download
 datatype: eeg
 basename: sub-002_task-eyesclosed_eeg.set),
 BIDSPath(
 root: ds004504-download
 datatype: eeg
 basename: sub-003_task-eyesclosed_eeg.set),
 BIDSPath(
 root: ds004504-download
 datatype: eeg
 basename: sub-004_task-eyesclosed_eeg.set),
 BIDSPath(
 root: ds004504-download
 datatype: eeg
 basename: sub-005_task-eyesclosed_eeg.set),
 BIDSPath(
 root: ds004504-download
 datatype: eeg
 basename: sub-006_task-eyesclosed_eeg.set),
 BIDSPath(
 root: ds004504-download
 datatype: eeg
 basename: sub-007_task-eyesclosed_eeg.set),
 BIDSPath(
 root: ds004504-download
 datatype: eeg
 basename: sub-008_task-eyesclosed_eeg.set),
 BIDSPath(
 root: ds004504-download
 datatype: eeg
 basename: sub-009_task-eyesclosed_eeg.set),
 BIDSPath(
 root: ds004504-download
 datatype: eeg
 basename: sub-010_task-eyesclosed_eeg.set),
 BIDSPath(
 root: ds004504-download
 dat

In [ ]:
sfreq = []
group = []
for subject_path in subject_paths:
    raw = mne_bids.read_raw_bids(subject_path)
    sfreq.append(raw.info['sfreq'])
    group.append(raw.info['subject_info']['Group'])
set(sfreq)
group

In [12]:
group[45]

'C'

## Extracting Features
### Edge Features

In [9]:
from scipy.signal import csd, hilbert, welch
from scipy import stats
from scipy.integrate import simps
import numpy as np
from abc import ABC, abstractmethod
from sklearn import preprocessing
import torch
from itertools import product
from torch_geometric.data import Data


In [10]:
SF = 500.0
NUM_CHANNELS = 19

In [11]:
def normalize(data: np.ndarray, dim=1, norm="l2") -> np.ndarray:
    """Normalizes the data channel by channel

    Args:
        data (np.ndarray): Raw EEG signal
        dim (int, optional): 0 columns, 1 rows. The dimension where 
        the mean and the std would be computed.

    Returns:
        np.ndarray: Channel-wise normalized matrix
    """
    normalized_data = preprocessing.normalize(data, axis=dim, norm=norm)
    return normalized_data

In [12]:
class BaseEdgeExtractor(ABC):
    
    @abstractmethod
    def __init__(self) -> None: ...
    
    @abstractmethod
    def extract_features(self, data) -> None: ...
        
    
    
class PearsonExtractor(BaseEdgeExtractor):
    def __init__(self, **kwargs) -> None:
        self.th = kwargs.get("th", None)
        self.normalize = kwargs.get("normalize", False)
    
    def extract_features(self, data) -> None:
        if self.normalize:
            data = normalize(data)
            
        corr_matrix = np.corrcoef(data)
        
        # Set to 0 values undes a
        if self.th:
            corr_matrix = np.where(
                abs(corr_matrix) < self.th, 0, corr_matrix
            )
        
        # Set to 0 self-loops
        corr_matrix = np.where(
            corr_matrix == 1, 0, corr_matrix
        )
        
        #corr_matrix = torch.from_numpy(corr_matrix)
        return corr_matrix
    
    
class PLIExtractor(BaseEdgeExtractor):
    def __init__(self, **kwargs) -> None:
        self.th = kwargs.get("th", None)
        self.normalize = kwargs.get("normalize", False)
        self.fs = kwargs.get("fs", 500.0)
        
        
    
    def extract_features(self, data) -> None:
        rows, cols = data.shape
        PLImatrix = np.zeros((rows, rows))    
        
        hilbert_transform = np.imag(hilbert(data))
        
        if self.normalize:
            data = normalize(data)
            
        for row in range(rows):
            for col in range(rows):
                PLImatrix[row, col] = np.abs(np.mean(np.exp(1j * (hilbert_transform[row] - hilbert_transform[col]))))  
          
        # Set to 0 values undes a th
        if self.th:
            PLImatrix = np.where(
                abs(PLImatrix) < self.th, 0, PLImatrix
            )
        
        # Remove self loops
        PLImatrix = np.where(
                abs(PLImatrix) == 1.0, 0, PLImatrix
            )
        
        #PLImatrix = torch.from_numpy(PLImatrix)
        return PLImatrix
    
    
class SpectralCoherenceExtractor(BaseEdgeExtractor):
    def __init__(self, **kwargs) -> None:
        self.th = kwargs.get("th", None)
        self.normalize = kwargs.get("normalize", False)
        
        
    
    def _compute_coherence(ch1, ch2, fs=SF):
        _, cross_spectral = csd(ch1, ch2,  fs=SF)
        coherence = np.abs(np.mean(cross_spectral)) / np.sqrt(np.mean(welch(ch1)) * np.mean(welch(ch2)))
        return coherence
    
    def extract_features(self, data) -> None:
        if self.normalize:
            data = normalize(data)
            
        
        coherence = [[np.abs(np.mean(csd(ch1, ch2, fs=256)[1])) / np.sqrt(np.mean(welch(ch1)) * np.mean(welch(ch2))) for ch2 in data] for ch1 in data]#[[ self._compute_coherence(channel1, channel2) for channel2 in data ] for channel1 in data]
        #coherence = [[np.abs(np.mean(csd(ch1, ch2, fs=256)[1]))  for ch2 in data] for ch1 in data]
        # Set to 0 values undes a
        if self.th:
            coherence = np.where(
                abs(coherence) < self.th, 0, coherence
            )
        
        # Set to 0 self-loops
        coherence = np.where(
            coherence == 1, 0, coherence
        )
        
        #coherence = torch.tensor(coherence)
        return coherence

### Node Features

In [13]:
#node features 
class BaseNodeExtractor(ABC):
    
    @abstractmethod
    def __init__(self) -> None: ...
    
    @abstractmethod
    def extract_features(self, data: np.ndarray): ...
    
    
class RawExtractor(BaseNodeExtractor):
    
    def __init__(self, **kwargs) -> None:
        super().__init__()
        self.normalize = kwargs.get("normalize", False)
        self.th = kwargs.get("th", None)
        
    def extract_features(self, data):
        if self.normalize:
            data = normalize(data)
            
        if self.th:
            data = np.where(
                abs(data) < self.th, 0, data
            )
            
        node_features = torch.from_numpy(data)
        return node_features
    
    
class StadisticalMomentsExtractor(BaseNodeExtractor):
    
    def __init__(self) -> None:
        """StadisticalMommentExtractor computes the: 
            - Mean
            - Std
            - Entropy
            - Variance
            - Skewness
            - Kurtosis
            
        For every channel independently and returns a tensor
        of NUMBER_CHANNELS x STATISTICAL_MOMENTS (19x6) 
        """
        super().__init__()
        
        
    def extract_features(self, data):
        mean = np.mean(data, axis=1)
        std = np.std(data, axis=1)
        variance = np.var(data, axis=1)
        entropy = stats.differential_entropy(data, axis=1)
        skewness = stats.skew(data, axis=1)
        kurtosis = stats.kurtosis(data, axis=1)
        node_features = np.array(
            [
                mean, std, variance, entropy, skewness, kurtosis
            ]
        ).T
        #node_features = torch.from_numpy(features)
        return node_features
    
    
class CWTExtractor(BaseNodeExtractor):
    
    def __init__(self) -> None:
        super().__init__()
        
        
    def extract_features(self, data):
        node_features = torch.from_numpy(data)
        return node_features
    
    
    
class PSDExtractor(BaseNodeExtractor):
    def __init__(self) -> None:
        super().__init__()
        self.bands = {
            "delta": [1, 4],
            "theta": [4, 7.5],
            "alpha": [7.5, 13],
            "lower_beta": [13, 16],
            "higher_beta": [16, 30], 
            "gamma": [30, 40]
        }
        
    def _bandpower(self, data, sf, band, window_sec=None, relative=True):
        """Compute the average power of the signal x in a specific frequency band.
        Parameters
        ----------
        data : 1d-array
            Input signal in the time-domain.
        sf : float
            Sampling frequency of the data.
        band : list
            Lower and upper frequencies of the band of interest.
        window_sec : float
            Length of each window in seconds.
            If None, window_sec = (1 / min(band)) * 2
        relative : boolean
            If True, return the relative power (= divided by the total power of the signal).
            If False (default), return the absolute power.

        Return
        ------
        bp : float
            Absolute or relative band power.
        
        This function its taken from: https://raphaelvallat.com/bandpower.html
        """
        band = np.asarray(band)
        low, high = band

        # Define window length
        if window_sec is not None:
            nperseg = window_sec * sf
        else:
            nperseg = (2 / low) * sf

        # Compute the modified periodogram (Welch)
        freqs, psd = welch(data, sf, nperseg=nperseg, nfft=2048)

        # Frequency resolution
        freq_res = freqs[1] - freqs[0]

        # Find closest indices of band in frequency vector
        idx_band = np.logical_and(freqs >= low, freqs <= high)

        # Integral approximation of the spectrum using Simpson's rule.
        bp = simps(psd[idx_band], dx=freq_res)

        if relative:
            bp /= simps(psd, dx=freq_res)
        
        
        return bp
        
    def extract_features(self, data):
        psds = [[self._bandpower(channel, SF, band) for band in self.bands.values()] for channel in data]
        #psds = torch.tensor(psds)
        return psds
    


### Graph Builder

In [14]:
class BaseGraphBuilder(ABC):
    
    @abstractmethod
    def __init__(self, normalize_nodes=False, normalize_edges=False) -> None: 
        self.node_feature_extractor = None
        self.edge_feature_extractor = None
        
    
    @abstractmethod
    def build(self, data, label):
        node_features = self.node_feature_extractor.extract_features(data)
        edge_features = self.edge_feature_extractor.extract_features(data)
        format_label = self._format_label(label)
        edge_index = torch.tensor(
            [[a, b] for a, b in product(range(NUM_CHANNELS), range(NUM_CHANNELS))]
        ).t().contiguous()
        
        graph = Data(
            x=node_features,
            edge_attr=edge_features,
            label=format_label,
            edge_index=edge_index
        )
        
        return graph
    
    def _onehot_label(self, label)-> torch.Tensor:
        if label == "A":
            return torch.tensor([[1, 0, 0]], dtype=torch.float64)
        if label == "C":
            return torch.tensor([[0, 1, 0]], dtype=torch.float64)
        if label == "F":
            return torch.tensor([[0, 0, 1]], dtype=torch.float64)
        
    def _format_label(self, label) -> torch.Tensor:
        label_dict = {"A": 1, "C": 0, "F": 2} # original: {"AD": 0, "HC": 1, "MCI": 2}
        return label_dict[label]
        
            
class RawAndPearson(BaseGraphBuilder):
    def __init__(self, normalize_nodes=False, normalize_edges=False, th=None) -> None:
        super().__init__()
        self.node_feature_extractor = RawExtractor(normalize=normalize_nodes)
        self.edge_feature_extractor = PearsonExtractor(normalize=normalize_edges, th=th)
        
        
    def build(self, data, label):
        return super().build(data, label)
    

class MomentsAndPearson(BaseGraphBuilder):
    def __init__(self, normalize_nodes=False, normalize_edges=False, th=None) -> None:
        super().__init__(normalize_nodes=normalize_nodes, normalize_edges=normalize_nodes)
        self.node_feature_extractor = StadisticalMomentsExtractor()
        self.edge_feature_extractor = PearsonExtractor(th=th)
        
    def build(self, data, label):
        return super().build(data, label)
    
    
class MomentsAndPLI(BaseGraphBuilder):
    def __init__(self, normalize_nodes=False, normalize_edges=False) -> None:
        super().__init__(normalize_nodes=normalize_nodes, normalize_edges=normalize_nodes)
        self.node_feature_extractor = StadisticalMomentsExtractor()
        self.edge_feature_extractor = PLIExtractor(th=0.1)
        
    def build(self, data, label):
        return super().build(data, label)
    
    
class RawAndPLI(BaseGraphBuilder):
    def __init__(self, normalize_nodes=False, normalize_edges=False) -> None:
        super().__init__(normalize_nodes=normalize_nodes, normalize_edges=normalize_nodes)
        self.node_feature_extractor = RawExtractor(normalize=normalize_nodes)
        self.edge_feature_extractor = PLIExtractor()
        
    def build(self, data, label):
        return super().build(data, label)
    
    
class PSDAndCSD(BaseGraphBuilder):
    def __init__(self, normalize_nodes=False, normalize_edges=False) -> None:
        super().__init__(normalize_nodes=normalize_nodes, normalize_edges=normalize_nodes)
        self.node_feature_extractor = PSDExtractor()
        self.edge_feature_extractor = SpectralCoherenceExtractor()
        
    def build(self, data, label):
        return super().build(data, label)
    
    
class PSDAndPearson(BaseGraphBuilder):
    def __init__(self, normalize_nodes=False, normalize_edges=False, th=None) -> None:
        super().__init__(normalize_nodes=normalize_nodes, normalize_edges=normalize_nodes)
        self.node_feature_extractor = PSDExtractor()
        self.edge_feature_extractor = PearsonExtractor(th=th)
        
    def build(self, data, label):
        return super().build(data, label)
    
    

class OfflineGeneric:
    
    def __init__(self, normalize_nodes=False, normalize_edges=False, th=None) -> None:
        super().__init__()
        self.node_feature_extractor = RawExtractor(normalize=normalize_nodes)
        self.edge_feature_extractor = RawExtractor(normalize=normalize_edges, th=th)
        

    def build(self, node_data, edge_data, label):
        node_features = self.node_feature_extractor.extract_features(node_data)
        edge_features = self.edge_feature_extractor.extract_features(edge_data)
        format_label = self._format_label(label)
        edge_index = torch.tensor(
            [[a, b] for a, b in product(range(NUM_CHANNELS), range(NUM_CHANNELS))]
        ).t().contiguous()
        
        graph = Data(
            x=node_features,
            edge_attr=edge_features,
            label=format_label,
            edge_index=edge_index
        )
        return graph
    
    def _format_label(self, label) -> torch.Tensor:
        label_dict = {"A": 0, "C": 1, "F": 2}
        return label_dict[label]

In [15]:
psd_pearson = PSDAndPearson()
PSD_PEARSON_PATH = Path('graphs/psd_pearson')
# Generating 1 graph
raw = mne_bids.read_raw_bids(subject_paths[0])
data = raw.get_data()
label = raw.info['subject_info']['Group']



Reading channel info from ds004504-download\sub-001\eeg\sub-001_task-eyesclosed_channels.tsv.


C:\Users\mashh\AppData\Local\Temp\ipykernel_18244\1363543725.py:4: RuntimeWarning: Data will be preloaded. preload=False or a string preload is not supported when the data is stored in the .set file
  raw = mne_bids.read_raw_bids(subject_paths[0])
C:\Users\mashh\AppData\Local\Temp\ipykernel_18244\1363543725.py:4: RuntimeWarning: Did not find any events.tsv associated with sub-001_task-eyesclosed.

The search_str was "ds004504-download\sub-001\**\eeg\sub-001*events.tsv"
  raw = mne_bids.read_raw_bids(subject_paths[0])


In [16]:
graph = psd_pearson.build(data, label)

In [17]:
graph

Data(x=[19], edge_index=[2, 361], edge_attr=[19, 19], label=1)

In [18]:
torch.save(graph, PSD_PEARSON_PATH.joinpath(Path(f'{str(subjects[0])}.pt')))


In [19]:
# counter = 0
# for subject_path in subject_paths:
#     raw = mne_bids.read_raw_bids(subject_path)
#     data = raw.get_data()
#     label = raw.info['subject_info']['Group']
#     graph = psd_pearson.build(data, label)
#     torch.save(graph, PSD_PEARSON_PATH.joinpath(Path(f'{str(subjects[counter])}.pt')))
#     counter += 1

In [23]:

builder_classes = [RawAndPearson(), MomentsAndPearson(), MomentsAndPLI(), RawAndPLI(), PSDAndCSD()]
graph_paths = [Path('graphs/raw_pearson'), Path('graphs/moments_pearson'), Path('graphs/moments_pli'), Path('graphs/raw_pli'),Path('graphs/psd_csd')]

for i in range(len(builder_classes)):
    graph_class = builder_classes[i]
    graph_path = graph_paths[i]
    
    for j in range(len(subject_paths)):
        raw = mne_bids.read_raw_bids(subject_paths[j])
        data = raw.get_data()
        label = raw.info['subject_info']['Group']
        graph = graph_class.build(data, label)
        if not os.path.exists(graph_path):
            os.makedirs(graph_path)
        torch.save(graph, graph_path.joinpath(f'{subjects[j]}.pt'))

    

Reading channel info from ds004504-download\sub-001\eeg\sub-001_task-eyesclosed_channels.tsv.


C:\Users\mashh\AppData\Local\Temp\ipykernel_18244\4265974583.py:9: RuntimeWarning: Data will be preloaded. preload=False or a string preload is not supported when the data is stored in the .set file
  raw = mne_bids.read_raw_bids(subject_paths[j])
C:\Users\mashh\AppData\Local\Temp\ipykernel_18244\4265974583.py:9: RuntimeWarning: Did not find any events.tsv associated with sub-001_task-eyesclosed.

The search_str was "ds004504-download\sub-001\**\eeg\sub-001*events.tsv"
  raw = mne_bids.read_raw_bids(subject_paths[j])


Reading channel info from ds004504-download\sub-002\eeg\sub-002_task-eyesclosed_channels.tsv.


C:\Users\mashh\AppData\Local\Temp\ipykernel_18244\4265974583.py:9: RuntimeWarning: Data will be preloaded. preload=False or a string preload is not supported when the data is stored in the .set file
  raw = mne_bids.read_raw_bids(subject_paths[j])
C:\Users\mashh\AppData\Local\Temp\ipykernel_18244\4265974583.py:9: RuntimeWarning: Did not find any events.tsv associated with sub-002_task-eyesclosed.

The search_str was "ds004504-download\sub-002\**\eeg\sub-002*events.tsv"
  raw = mne_bids.read_raw_bids(subject_paths[j])


Reading channel info from ds004504-download\sub-003\eeg\sub-003_task-eyesclosed_channels.tsv.


...

C:\Users\mashh\AppData\Local\Temp\ipykernel_18244\4265974583.py:9: RuntimeWarning: Data will be preloaded. preload=False or a string preload is not supported when the data is stored in the .set file
  raw = mne_bids.read_raw_bids(subject_paths[j])
C:\Users\mashh\AppData\Local\Temp\ipykernel_18244\4265974583.py:9: RuntimeWarning: Did not find any events.tsv associated with sub-086_task-eyesclosed.

The search_str was "ds004504-download\sub-086\**\eeg\sub-086*events.tsv"
  raw = mne_bids.read_raw_bids(subject_paths[j])


Reading channel info from ds004504-download\sub-087\eeg\sub-087_task-eyesclosed_channels.tsv.


C:\Users\mashh\AppData\Local\Temp\ipykernel_18244\4265974583.py:9: RuntimeWarning: Data will be preloaded. preload=False or a string preload is not supported when the data is stored in the .set file
  raw = mne_bids.read_raw_bids(subject_paths[j])
C:\Users\mashh\AppData\Local\Temp\ipykernel_18244\4265974583.py:9: RuntimeWarning: Did not find any events.tsv associated with sub-087_task-eyesclosed.

The search_str was "ds004504-download\sub-087\**\eeg\sub-087*events.tsv"
  raw = mne_bids.read_raw_bids(subject_paths[j])


Reading channel info from ds004504-download\sub-088\eeg\sub-088_task-eyesclosed_channels.tsv.


C:\Users\mashh\AppData\Local\Temp\ipykernel_18244\4265974583.py:9: RuntimeWarning: Data will be preloaded. preload=False or a string preload is not supported when the data is stored in the .set file
  raw = mne_bids.read_raw_bids(subject_paths[j])
C:\Users\mashh\AppData\Local\Temp\ipykernel_18244\4265974583.py:9: RuntimeWarning: Did not find any events.tsv associated with sub-088_task-eyesclosed.

The search_str was "ds004504-download\sub-088\**\eeg\sub-088*events.tsv"
  raw = mne_bids.read_raw_bids(subject_paths[j])


In [13]:
def extract_features(self, data):
        mean = np.mean(data, axis=1)
        std = np.std(data, axis=1)
        variance = np.var(data, axis=1)
        entropy = stats.differential_entropy(data, axis=1)
        skewness = stats.skew(data, axis=1)
        kurtosis = stats.kurtosis(data, axis=1)
        node_features = np.array(
            [
                mean, std, variance, entropy, skewness, kurtosis
            ]
        ).T
        #node_features = torch.from_numpy(features)
        return node_features


In [14]:
raw

<RawEEGLAB | sub-088_task-eyesclosed_eeg.set, 19 x 397050 (794.1 s), ~57.6 MB, data loaded>

In [15]:
raw.get_data

<bound method get_data of <RawEEGLAB | sub-088_task-eyesclosed_eeg.set, 19 x 397050 (794.1 s), ~57.6 MB, data loaded>>

In [16]:
raw.get_data()

array([[ 1.07080078e-04,  1.25244133e-04,  1.29492187e-04, ...,
        -2.38183578e-04, -2.36621078e-04, -2.19628891e-04],
       [ 2.85644493e-05,  4.25292931e-05,  4.32617149e-05, ...,
        -2.23730453e-04, -2.36230453e-04, -2.23632797e-04],
       [ 6.88476944e-06,  2.07519569e-05,  2.30468788e-05, ...,
        -1.06542969e-04, -1.02343750e-04, -8.31054687e-05],
       ...,
       [-7.35839844e-05, -5.66894493e-05, -5.08789024e-05, ...,
        -1.19824203e-04, -1.20800766e-04, -1.02832016e-04],
       [-8.13964844e-05, -6.46972656e-05, -5.81054649e-05, ...,
        -1.36914047e-04, -1.36718735e-04, -1.18652328e-04],
       [-1.27490219e-04, -1.11669914e-04, -1.05664055e-04, ...,
        -9.94140625e-05, -9.87304687e-05, -7.97851562e-05]])

In [21]:
import numpy as np
from scipy import stats
data = raw.get_data()
mean = np.mean(data, axis=1)
std = np.std(data, axis=1)
variance = np.var(data, axis=1)
entropy = stats.differential_entropy(data, axis=1)
skewness = stats.skew(data, axis=1)
kurtosis = stats.kurtosis(data, axis=1)
node_features = np.array(
    [
        mean, std, variance, entropy, skewness, kurtosis
    ]
).T

In [23]:
node_features

array([[-9.71431121e-05,  8.60768060e-05,  7.40921654e-09,
        -7.96441362e+00, -4.00075059e-01,  1.56738464e+00],
       [-7.88811182e-05,  1.08017724e-04,  1.16678287e-08,
        -7.72636043e+00, -2.78537584e-01,  5.40558448e-01],
       [ 5.37647576e-06,  7.20989805e-05,  5.19826298e-09,
        -8.12062834e+00,  1.03966266e-01,  1.24260303e-01],
       [ 1.43127033e-06,  6.78771033e-05,  4.60730115e-09,
        -8.18048683e+00,  1.83349287e-02,  4.45953540e-02],
       [-1.64687748e-05,  6.57647292e-05,  4.32499961e-09,
        -8.21354242e+00, -1.05837417e-01,  3.37058232e-03],
       [ 1.41801665e-05,  6.56732232e-05,  4.31297224e-09,
        -8.21440755e+00,  5.63115117e-02,  2.60147587e-01],
       [-1.77536653e-05,  7.12666529e-05,  5.07893581e-09,
        -8.14091671e+00,  3.24651861e-01,  6.15571151e-01],
       [ 3.72979672e-05,  6.95074464e-05,  4.83128510e-09,
        -8.16542671e+00,  3.18301096e-01,  5.10420092e-01],
       [ 2.95964468e-05,  7.97217571e-05,  6.355

In [24]:
data.shape


(19, 397050)